In [4]:
import numpy as np

In [5]:
class HGSO:
    def __init__(self):
        self.l1 = 5e-2
        self.l2 = 100
        self.l3 = 1e-2
        self.maximum_optimization = 1
        self.t0 = 298.15
        self.K = 1
        self.epsilon=0.05
        self.c1 = 0.1
        self.c2 = 0.2
        self.compiled = 0
        self.population = None
        self.fitnesses = None
        self.X_best = None
        self.Xi_best = None
        self.best_fitness = None
        
    def compile(self,obj,lb,ub,dim,n_population=100,n_cluster=5,alpha=1.0,beta=1.0,optimization="maximize"):
        if n_population%n_cluster != 0:
            raise Exception("Number of search agent in each cluster shoudl be same")
        self.dim = dim
        self.lb = lb
        self.ub = ub
        self.alpha = alpha
        self.beta = beta
        self.n_population = n_population
        self.n_cluster = n_cluster
        self.optimization = optimization
        self.comp_id = 0 if optimization == "maximize" else -1
        self.obj = obj
        self.compiled = 1
    
    def __get_fitness(self,X):
        if self.optimization == "maximize":
            return self.obj(X)
        elif self.optimization == "minimize":
            return 1/(self.obj(X)+0.002)
        
    def __initialize_population(self,dim,n_population,n_cluster):
        population = np.random.uniform(self.lb,self.ub,size=(self.n_population,self.dim))
        fitness = []
        for x in population:
            fitness.append(self.__get_fitness(x))
        return population,np.array(fitness)
    
    def __initialize_constant(self,n_population,n_cluster):
        h_j = (self.l1 * np.random.uniform(0,1,n_cluster)).reshape(-1,1)
        C_j = (self.l3 * np.random.uniform(0,1,n_cluster)).reshape(-1,1)
        n_j = n_population//n_cluster
        P_ij = self.l2 * np.random.uniform(0,1,size=(n_cluster,n_j))
        return h_j,C_j,P_ij
    
    def __get_cluster_best(self,population_fitness,n_cluster):
        n_element = self.n_population//n_cluster
        cluster_best = []
        
        for i in range(n_cluster):
            li = i*n_element
            ui = li + n_element
            best = np.argsort(population_fitness[li:ui])[-1]
            cluster_best.append(i*n_element+best)
        return np.array(cluster_best)
    
    def __get_population_best(self,population_fitness):
        return self.__get_cluster_best(population_fitness,1)
        
    def __update_h_j(self,max_iter,t):
        T = t/max_iter
        h_j = self.h_j * np.exp(-self.C_j*((1.0/T)-(1.0/self.t0)))
        
        return h_j
    
    def __update_S_ij(self,h_j,P_ij):
        S_ij = self.K * h_j * P_ij
        
        return S_ij
    
    def __update_worst_agent(self):
        #Getting the value of N_w
        N_w = int(self.n_population * (np.random.uniform(0,self.c2-self.c1)+self.c1))
        # Getting the index of worst agent in the population
        
        worst_pop_index = np.argsort(self.fitnesses)[:N_w]
        #print(f"N_w: {N_w}")
        # Updating the worst agent in the population
        self.population[worst_pop_index] = np.random.uniform(self.lb,self.ub,size=(N_w,self.dim))
        # Updating the fitnesses of newly assigned agent
        for i in worst_pop_index:
            self.fitnesses[i] = self.__get_fitness(self.population[i])
    
    def __update_global_best(self):
        best_id = self.__get_population_best(self.fitnesses)
        print(best_id)
        if self.best_fitness == None or self.best_fitness < self.fitnesses[best_id]:
            self.X_best = self.population[best_id]
            self.best_fitness = self.fitnesses[best_id]
        
    def __update_population(self):
        # Setting the F value according to a random number to make the update diverse
        F = []
        for i  in range(self.n_population):
            f = -1 if np.random.uniform() < 0.5 else 1
            F.append(f)
        F = np.array(F).reshape(-1,1)
        #print(F.shape)
        #print("Hello World")
        
        # r is a random value
        r = np.random.uniform(0,1,size=(self.n_population,1))
        
        # Getting the number of population in a clusters
        n_cluster_population = self.n_population//self.n_cluster
        
        #Getting the best search agent in cluster and best agent in the whole population
        X_best = self.X_best
        Xi_best = self.population[self.Xi_best].repeat(n_cluster_population,axis=0)
        
        # Getting the value of constant  gamma
        epsilon = 0.05
        best_fitnesses = self.fitnesses[self.Xi_best].repeat(n_cluster_population)

        gamma = self.beta * np.exp(-(best_fitnesses+self.epsilon)/(self.fitnesses+self.epsilon))
        gamma = gamma.reshape(-1,1)
        updated_population = self.population * F * r * gamma * (Xi_best-self.population) + F * r * self.alpha * (self.S_ij.reshape(-1,1) * X_best + self.population)
        updated_population = np.clip(updated_population,self.lb,self.ub)
        #print(gamma.shape)
        fitnesses = []
        for i in updated_population:
            fitnesses.append(self.__get_fitness(i))
        return updated_population,np.array(fitnesses)
    
    def train(self,max_iter):
        # Checking if compiled
        if self.compiled == 0:
            raise Exception("You have to compile first.")
        # Initializing the population and cluster, calculating the fitness of the populations
        self.population,self.fitnesses= self.__initialize_population(self.dim,self.n_population,self.n_cluster)
        
        # Intializing the Necessary constant
        self.h_j,self.C_j,self.P_ij = self.__initialize_constant(self.n_population,self.n_cluster)
        # Getting the fitness and getting the best gasses in the clusters and in the swarm
        
        # Getting the index of best search agent in population and clusters
        #self.X_best = self.__get_population_best(self.fitnesses)
        #self.Xi_best = self.__get_cluster_best(self.fitnesses,self.n_cluster)
        
        for i in range(max_iter):
            
            # Evaluating the current populations using fitnesses
            self.__update_global_best()
            
            self.Xi_best = self.__get_cluster_best(self.fitnesses,self.n_cluster)
            
            # Updating the Henry Gas Constant for each cluster
            self.h_j = self.__update_h_j(max_iter,i+1)

            # Updating the solubility of the gas
            self.S_ij = self.__update_S_ij(self.h_j,self.P_ij)
            self.population,self.fitnesses = self.__update_population()
            
            # Updating the position of worst agent in the population
            self.__update_worst_agent()
            print(f"Best Agent: {self.X_best}\t Best Value: {self.best_fitness}")

In [6]:
hgso = HGSO()

In [7]:
def Sphere_func(x):
    fitness = 0.0
    for i in range(len(x)):
        fitness += (x[i]*x[i])
    return fitness

dim = 3
lb = np.array([-10.0,-10,-10],dtype='f')
ub = np.array([10.0,10.0,10.0],dtype='f')


In [8]:
hgso.compile(Sphere_func,lb,ub,dim,optimization="minimize")

In [9]:
hgso.train(10)

[16]
Best Agent: [[-0.95527444  1.77548874 -1.40119956]]	 Best Value: [0.16583006]
[63]
Best Agent: [[ 0.09573318 -0.02409781  0.00418677]]	 Best Value: [85.01178545]
[89]
Best Agent: [[0.00098495 0.01390411 0.00179447]]	 Best Value: [455.05957263]
[78]
Best Agent: [[-0.00304688 -0.00249012 -0.00064321]]	 Best Value: [496.05686204]
[99]
Best Agent: [[ 0.00057737  0.00017905 -0.00043927]]	 Best Value: [499.86044634]
[90]
Best Agent: [[ 1.89721714e-04 -5.85245246e-05  3.35412462e-05]]	 Best Value: [499.98986409]
[79]
Best Agent: [[-4.21574340e-05 -2.15531731e-07  4.59416814e-06]]	 Best Value: [499.9995504]
[89]
Best Agent: [[-5.27241517e-07  7.25015192e-06  1.35900981e-06]]	 Best Value: [499.99998633]
[89]
Best Agent: [[-1.24176685e-07  1.70756627e-06  3.20075957e-07]]	 Best Value: [499.99999924]
[13]
Best Agent: [[-4.37064829e-07 -4.16793379e-07 -1.08889670e-06]]	 Best Value: [499.99999961]


In [56]:
import numpy as np
from numpy.random import uniform
from copy import deepcopy

def Sphere_func(x):
    fitness = 0.0
    for i in range(len(x)):
        fitness += (x[i]*x[i])
    return fitness
class HGSO():

	ID_MIN_PROB = 0 # min problem
	ID_MAX_PROB = -1 # max problem
	ID_POS = 0 # Position
	ID_FIT = 1 # Fitness

	def __init__(self, obj_func=None, lb=None, ub=None,
				verbose=True, epoch=750, pop_size=100,
				n_clusters=2, **kwargs):
		self.epoch = epoch
		self.pop_size = pop_size
		self.n_clusters = n_clusters
		self.n_elements = int(self.pop_size / self.n_clusters)
		self.lb = lb
		self.ub = ub
		self.verbose = verbose
		self.T0 = 298.15
		self.K = 1.0
		self.beta = 1.0
		self.alpha = 1
		self.epxilon = 0.05
		self.obj_func = obj_func
		self.l1 = 5E-2
		self.l2 = 100.0
		self.l3 = 1E-2
		self.H_j = self.l1 * uniform()
		self.P_ij = self.l2 * uniform()
		self.C_j = self.l3 * uniform()
		self.solution, self.loss_train = None, []

	def get_fitness_position(self, position=None, minmax=0):
		return self.obj_func(position) if minmax == 0 else 1.0 / (
		self.obj_func(position) + 10E-10)

	def get_fitness_solution(self, solution=None, minmax=0):
		return self.get_fitness_position(solution[self.ID_POS], minmax)

	def get_global_best_solution(self, pop=None, id_fit=None, id_best=None):
	
		# Sort a copy of population and return the copy of
		# the best position
		sorted_pop = sorted(pop, key=lambda temp: temp[id_fit])
		return deepcopy(sorted_pop[id_best])

	def update_global_best_solution(self, pop=None, id_best=None, g_best=None):
	
		# Sort the copy of population and update the current best
		# position. Return the new current best position """
		sorted_pop = sorted(pop, key=lambda temp: temp[self.ID_FIT])
		current_best = sorted_pop[id_best]
		return deepcopy(current_best) if current_best[self.ID_FIT] <\
	g_best[self.ID_FIT] else deepcopy(g_best)

	def create_population__(self, minmax=0, n_clusters=0):
		pop = []
		group = []
		
		for i in range(n_clusters):
			team = []
			
			for j in range(self.n_elements):
				solution = uniform(self.lb, self.ub)
				fitness = self.obj_func(
					solution) if minmax == 0 else 1.0 / (
				self.obj_func(solution) + 10E-10)
				
				team.append([solution, fitness, i])
				pop.append([solution, fitness, i])
			group.append(team)
		return pop, group

	def get_best_solution_in_team(self, group=None):
		list_best = []
		
		for i in range(len(group)):
			sorted_team = sorted(group[i], key=lambda temp: temp[self.ID_FIT])
			list_best.append(deepcopy(sorted_team[self.ID_MIN_PROB]))
		return list_best

	def train(self):
		pop, group = self.create_population__(
			self.ID_MIN_PROB, self.n_clusters)
		
		# single element
		g_best = self.get_global_best_solution(
			pop, self.ID_FIT, self.ID_MIN_PROB)
		
		# multiple element
		p_best = self.get_best_solution_in_team(
			group)

		# Loop iterations
		for epoch in range(self.epoch):

			# Loop based on the number of cluster in swarm
			# number of gases type)
			for i in range(self.n_clusters):

				# Loop based on the number of individual in
				# each gases type
				for j in range(self.n_elements):

					F = -1.0 if uniform() < 0.5 else 1.0

					# Based on Eq. 8, 9, 10
					self.H_j = self.H_j * \
						np.exp(-self.C_j *
							(1.0/np.exp(-epoch/self.epoch) - 1.0/self.T0))
					S_ij = self.K * self.H_j * self.P_ij
					gamma = self.beta * \
						np.exp(- ((p_best[i][self.ID_FIT] + self.epxilon) /
								(group[i][j][self.ID_FIT] + self.epxilon)))

					X_ij = group[i][j][self.ID_POS] + F * uniform() * gamma * \
					(p_best[i][self.ID_POS] - group[i][j][self.ID_POS]) + \
						F * uniform() * self.alpha * \
						(S_ij * g_best[self.ID_POS] - group[i][j][self.ID_POS])

					fit = self.get_fitness_position(X_ij, self.ID_MIN_PROB)
					group[i][j] = [X_ij, fit, i]
					pop[i*self.n_elements + j] = [X_ij, fit, i]

			# Update Henry's coefficient using Eq.8
			self.H_j = self.H_j * \
				np.exp(-self.C_j * (1.0 / np.exp(-epoch / self.epoch) - 1.0 / self.T0))
			
			# Update the solubility of each gas using Eq.9
			S_ij = self.K * self.H_j * self.P_ij
			
			# Rank and select the number of worst agents using Eq. 11
			N_w = int(self.pop_size * (uniform(0, 0.1) + 0.1))
			
			# Update the position of the worst agents using Eq. 12
			sorted_id_pos = np.argsort([x[self.ID_FIT] for x in pop])

			for item in range(N_w):
				id = sorted_id_pos[item]
				j = id % self.n_elements
				i = int((id-j) / self.n_elements)
				X_new = uniform(self.lb, self.ub)
				fit = self.get_fitness_position(X_new, self.ID_MIN_PROB)
				pop[id] = [X_new, fit, i]
				group[i][j] = [X_new, fit, i]

			p_best = self.get_best_solution_in_team(group)
			g_best = self.update_global_best_solution(
				pop, self.ID_MIN_PROB, g_best)
			self.loss_train.append(g_best[self.ID_FIT])
			
			if self.verbose:
				print("Epoch: {}, Best fitness value: {}".format(
					epoch + 1, g_best[self.ID_FIT]))
		self.solution = g_best
		return g_best[self.ID_POS], g_best[self.ID_FIT], self.loss_train


lb = [-10]
ub = [10]
epoch = 100
verbose = True
pop_size = 50
obj = HGSO(Sphere_func, lb, ub, verbose, epoch, pop_size)
obj.train()


Epoch: 1, Best fitness value: 0.01305222135034654
Epoch: 2, Best fitness value: 0.0014877703452542393
Epoch: 3, Best fitness value: 0.0014877703452542393
Epoch: 4, Best fitness value: 0.0014877703452542393
Epoch: 5, Best fitness value: 0.0014877703452542393
Epoch: 6, Best fitness value: 0.0014877703452542393
Epoch: 7, Best fitness value: 0.0014877703452542393
Epoch: 8, Best fitness value: 0.0014877703452542393
Epoch: 9, Best fitness value: 0.0014877703452542393
Epoch: 10, Best fitness value: 0.0014877703452542393
Epoch: 11, Best fitness value: 0.0014877703452542393
Epoch: 12, Best fitness value: 0.0014877703452542393
Epoch: 13, Best fitness value: 0.0014877703452542393
Epoch: 14, Best fitness value: 0.0014877703452542393
Epoch: 15, Best fitness value: 0.0014877703452542393
Epoch: 16, Best fitness value: 0.0014877703452542393
Epoch: 17, Best fitness value: 0.0014877703452542393
Epoch: 18, Best fitness value: 1.9666923481563715e-05
Epoch: 19, Best fitness value: 1.9666923481563715e-05
Ep

(array([-0.00251209]),
 6.310605037906587e-06,
 [0.01305222135034654,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  0.0014877703452542393,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.9666923481563715e-05,
  1.966692

In [ ]:
sphere_func(x):
    X = [x1,x2,x3,x4]
    return x1**2+x2**2+x3**2+x4**2

In [3]:

class Sphere():
    def __init__(self,dim,opt_value):
        self.dim=dim
        self.opt_value = opt_value
        self.optimization = 'minimize'
    
    def get_algorithm(self):
        def sphere_func(x):
            res=0
            for i in x:
                res+=i**2
            return res
        return sphere_func
    
sphere = Sphere(3,0)
print(sphere.get_algorithm())

<function Sphere.get_algorithm.<locals>.sphere_func at 0x7f2354464820>
